In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import importlib
from datetime import datetime, timedelta
import requests
import sys
import os
import logging
import time
import pytz

#from coinmetrics.api_client import CoinMetricsClient
from dateutil.relativedelta import relativedelta

#Talos
sys.path.append(os.path.expanduser("~") + "/anchorage/source/python/trading/agency_desk/lib/")
import talos_utils

#Coinmetrics
path = os.path.expanduser("~") + "/jupyter-server/transferapi/"
sys.path.append(path)
#import transferapi_lib
key_folder = path + "./key/"
#key_file = key_folder + "coin_metrics.key"
#coinmetrics_key, null_key = transferapi_lib.load_revenue_key(key_file)
#client = CoinMetricsClient(coinmetrics_key, verbose=False)

## Talos Env ##
env = "x"
##############

if env == "wl":
    talos_adv_api = 'ANC0ASVTOGM0'
    talos_adv_secret_api = '8myg2ro5bf2pzfx4m79u9yh501e09nky'
    host_wl = "tal-160.prod.talostrading.com"
    talos = talos_utils.Talos(talos_adv_api, talos_adv_secret_api, host_wl)   
else:   
    talos_api = 'ANC80D4RI4DL'
    talos_secret = 'mv1ahb471yqfij1lt9g8bamhokibonwq'
    host = "tal-42.prod.talostrading.com"
    talos = talos_utils.Talos(talos_api, talos_secret, host)

In [2]:
def multi_order_fills(all_orders):
    combined_df = pd.DataFrame()  # Initialize an empty dataframe to store the combined data
    for identifier in all_orders:
        # Call run_function for each identifier and get the dataframe
        try:
            df = talos.get_trade_fills(order_id=identifier)
            df['Amount'] = df['Amount'].astype(float)
            df['Fee'] = df['Fee'].astype(float)
            df['Quantity'] = df['Quantity'].astype(float)
            df["amount_less_fees"] = df["Amount"] - df["Fee"]
            # Concatenate the dataframe to the combined dataframe
            combined_df = pd.concat([combined_df, df], ignore_index=True)
            time.sleep(0.100)
        except Exception as e:
            err_str = f"ERROR during get_trade_fills for orderID {identifier}: {str(e)}\n\n"
            print(err_str)
    if "Timestamp" in combined_df.columns:
        combined_df = combined_df.set_index('Timestamp')
        combined_df = combined_df.sort_index()
    return combined_df

def fills_by_dealer(df, asset, dealer_cutoff_start=None, dealer_cutoff_end=None):
    if dealer_cutoff_start and dealer_cutoff_end is not None:
        filtered_df = df[(df.index >= dealer_cutoff_start) & (df.index < dealer_cutoff_end)]
    else:
        filtered_df = df
    # Group by dealer
    df_by_dealer = filtered_df.groupby("Market")[["amount_less_fees", "Quantity"]].sum().reset_index()
    df_count = filtered_df.groupby("Market")["OrderID"].count().reset_index()
    df_count = df_count.rename(columns={"OrderID": "fill_count"})
    
    pd.options.display.float_format = '{:,.8f}'.format
    amount = filtered_df['amount_less_fees'].sum()
    qty = filtered_df['Quantity'].sum()
    exec_price = amount / qty

    summary_str = f"USD Amount: ${amount:,.2f}\nQuantity: {qty:,.4f} {asset}\nExecution Price: ${exec_price:,.2f}\n"
    summary_df = df_by_dealer.merge(df_count)
    return filtered_df, summary_str, summary_df, exec_price


def vwap(x):
    total_volume = x["amount"].sum()
    return (x['price'] * x["amount"]).sum() / total_volume

def vwap_talos(x):
    total_volume = x["Size"].sum()
    return (x['Price'] * x["Size"]).sum() / total_volume

def add_calculations(df):
    df = df.copy()
    df["vwas"] = (df["Quantity"] * df["bps_spread_to_ref"]).cumsum() / df["Quantity"].cumsum()
    df["cum_qty"] = df["Quantity"].cumsum()
    
    df['rolling_avg_spread'] = df['bps_spread_to_ref'].rolling(len(df), min_periods=1).mean()
    return df

def resample_exchange_trades(exchange_trades):
    # Convert column name to match Talos API output and set as index
    exchange_trades = exchange_trades.rename(columns={"time": "Timestamp"})
    exchange_trades = exchange_trades.set_index('Timestamp').copy()
    
    # Convert timestamp from UTC to EST 
    exchange_trades.index = exchange_trades.index.tz_convert("US/Eastern").tz_localize(None)
    
    # Resample CB trades dataset to VWAP price across each 1S interval. Fill forward where there are no trades for an interval
    exchange_trades_resampled = exchange_trades.resample("500ms").apply(vwap)
    exchange_trades = pd.DataFrame(exchange_trades_resampled, columns=["vwap"])
    exchange_trades = exchange_trades.ffill()
    
    return exchange_trades
    
def calculate_spread_to_cb(all_dealer_fills, exchange_trades):
    # Convert timestamps to int64 (idk if this is needed but I was getting errors)
    all_dealer_fills.index.astype(np.int64) 
    exchange_trades.index.astype(np.int64) 

    # Merge dealer fills df with CB 1s interval VWAP df
    merged_df = pd.merge_asof(all_dealer_fills, exchange_trades, left_index=True, right_index=True, direction='nearest')
    
    # Calculate spread of fills to CB in bps
    cols = ['PriceAllIn', 'vwap', 'Quantity']
    merged_df[cols] = merged_df[cols].apply(pd.to_numeric, errors='coerce', axis=1)
    merged_df["bps_spread_to_ref"] = ((merged_df["PriceAllIn"] - merged_df["vwap"]) / merged_df["vwap"]) * 10000
    merged_df["bps_spread_to_ref"] = pd.to_numeric(merged_df["bps_spread_to_ref"], errors='coerce')

    # Calculate Average and Volume weighted Spread for all fills
    avg_spread_to_ref = merged_df['bps_spread_to_ref'].mean()
    total_volume = merged_df['Quantity'].sum()
    vwas = (merged_df['bps_spread_to_ref'] * merged_df['Quantity']).sum() / total_volume
    summary_str = f"Mean Spread to Reference px: {avg_spread_to_ref:,.4f} bps\nVolume-weighted Avg Spread to Reference px: {vwas:,.4f} bps\n"
   
    return merged_df, summary_str

def datetime_to_str(timestamp):
    utc_datetime = pytz.timezone('US/Eastern').localize(timestamp).astimezone(pytz.utc)
    utc_string = utc_datetime.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
    
    return utc_string

def convert_to_et(datetime_string):
    datetime_obj = datetime.strptime(datetime_string, "%Y-%m-%dT%H:%M:%S.%fZ")
    eastern = pytz.timezone('US/Eastern')
    datetime_et = pytz.utc.localize(datetime_obj).astimezone(eastern)
    
    return datetime_et.replace(tzinfo=None)

def get_exchange_trades_df(symbol, market, startDate, endDate):
    # get talos market data, convert to df
    trades = talos.get_market_trades(symbol, market, startDate, endDate)
    df = pd.DataFrame(trades)
    # set timestamp as index, convert to EST
    df['TransactTime'] = df['TransactTime'].apply(lambda x: convert_to_et(x))
    df = df.drop(columns=['Timestamp'])
    
    return df

def resample_exchange_trades_talos(exchange_trades):
    # Convert column name to match Talos API output and set as index
    exchange_trades = exchange_trades.rename(columns={"TransactTime": "Timestamp"})
    exchange_trades = exchange_trades.set_index('Timestamp').copy()
    
    # Resample CB trades dataset to VWAP price across each 1S interval. Fill forward where there are no trades for an interval
    exchange_trades["Price"] = exchange_trades["Price"].astype(float)
    exchange_trades["Size"] = exchange_trades["Size"].astype(float) 
    exchange_trades_resampled = exchange_trades.resample("500ms").apply(vwap_talos)
    exchange_trades = pd.DataFrame(exchange_trades_resampled, columns=["vwap_talos"])
    exchange_trades = exchange_trades.ffill()
    
    return exchange_trades

def calculate_spread_to_ref(all_dealer_fills, exchange_trades):
    # Merge dealer fills df with CB 1s interval VWAP df
    merged_df = pd.merge_asof(all_dealer_fills, exchange_trades, left_index=True, right_index=True, direction='nearest')
    
    # Calculate spread of fills to CB in bps
    cols = ['PriceAllIn', 'vwap_talos', 'Quantity']
    merged_df[cols] = merged_df[cols].apply(pd.to_numeric, errors='coerce', axis=1)
    merged_df["bps_spread_to_ref"] = ((merged_df["PriceAllIn"] - merged_df["vwap_talos"]) / merged_df["vwap_talos"]) * 10000
    merged_df["bps_spread_to_ref"] = pd.to_numeric(merged_df["bps_spread_to_ref"], errors='coerce')

    # Calculate Average and Volume weighted Spread for all fills
    avg_spread_to_ref = merged_df['bps_spread_to_ref'].mean()
    total_volume = merged_df['Quantity'].sum()
    vwas = (merged_df['bps_spread_to_ref'] * merged_df['Quantity']).sum() / total_volume
    summary_str = f"Mean Spread to Reference px: {avg_spread_to_ref:,.4f} bps\nVolume-weighted Avg Spread to Reference px: {vwas:,.4f} bps\n"
    
    return merged_df, summary_str

In [6]:
## LIVE ORDER DATA ##
# Add OrderID & date filters"
all_orders = ["bb663745-b76e-4f56-9f86-7b765c1d537f",
"effd4a96-fc28-4189-9b1b-e4335e476d71",
"544484a3-a584-416f-afac-80258d3e7fa7",
"283f296c-0f98-4ab5-ab77-f4f8548a95fc",
"d6b16a5c-0454-43ff-946e-8514f53c52e5",
"c00d865b-c86c-4b71-ba35-d98f627bca23",
"4278af32-ad52-4e3c-80cc-4b43611f4d63",
"b850a4f2-14a2-4040-bc24-8dac9f580bb4"]

# datetime(year, month, day, hour, minute, second, microsecond) IN EST!!!
dealer_cutoff_start = datetime(2024, 9, 18, 9, 0, 0, 0)
dealer_cutoff_end = datetime(2024, 9, 19, 13, 0, 0, 0)

# Run fill data for each order in list
all_dealer_fills = multi_order_fills(all_orders)
filtered_df, summary_str, summary_df, exec_price = fills_by_dealer(all_dealer_fills, "BTC", dealer_cutoff_start, dealer_cutoff_end)
# filtered_df, summary_str, summary_df = fills_by_dealer(combined_df)

print(summary_str)
summary_df

USD Amount: $237,137,649.78
Quantity: 3,879.3415 BTC
Execution Price: $61,128.32



,Market,amount_less_fees,Quantity,fill_count
0,cumberland,"116,620,952.63917500","1,911.29260102",4114
1,janestreet,"92,407,161.95429581","1,499.96042852",12985
2,wintermute,"28,109,535.19000000",468.08850902,677


In [18]:
#update host for Talos market data endpoint and re-initialize talos_utils
host_market_data = "talostrading.com"
talos_adv_api = 'ANC8QX7D6WRE'
talos_adv_secret_api = 'nqxfvg2xg9sjpdglo5ul5ydepzpv3h9r'
talos = talos_utils.Talos(talos_adv_api, talos_adv_secret_api, host_market_data)

# get data from order fills df
symbol = all_dealer_fills.Symbol[0]
first_fill = all_dealer_fills.index[0].to_pydatetime() - timedelta(minutes=0.1)
last_fill = all_dealer_fills.index.max().to_pydatetime() + timedelta(minutes=0.5)

# enter params for market data
market = "coinbase,gemini,kraken"
startDate = datetime_to_str(first_fill)
endDate = datetime_to_str(last_fill)

# get exchange trades
exchange_trades = get_exchange_trades_df(symbol, market, startDate, endDate)
exchange_trades_resampled = resample_exchange_trades_talos(exchange_trades)

/tmp/ipykernel_1055334/3792716168.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  symbol = all_dealer_fills.Symbol[0]


AttributeError: 'Talos' object has no attribute 'get_market_trades'

In [19]:
merged_df, spread_summary_str = calculate_spread_to_ref(all_dealer_fills, exchange_trades_resampled)
spread_summary_str

NameError: name 'exchange_trades_resampled' is not defined

### Ad Hoc Analysis

In [ ]:
df = pd.read_csv('COINBASE_MKRUSD, 1S - Sheet1.csv')
exchange_df = df[['time', 'close']].copy()
exchange_df["time"] = pd.to_datetime(exchange_df["time"])
exchange_df = exchange_df.set_index('time')
exchange_df.index = exchange_df.index.tz_convert("US/Eastern").tz_localize(None)

In [ ]:
exchange_df.head()

In [ ]:
order_fills = all_dealer_fills[['Price', 'Quantity']].copy()
order_fills

In [ ]:
merged_df = pd.merge_asof(order_fills, exchange_df, left_index=True, right_index=True, direction='forward')
merged_df

In [ ]:
file_name = '/home/michael_marano_anchorlabs_com/Quant_Research/csv_files/tca_test_mkr.csv'
merged_df.to_csv(file_name)

In [ ]:
## LIVE ORDER DATA ##
# Add OrderID & date filters
all_orders = ["0cc01f4e-d5bd-436a-80a6-d90f5807d3c2",
             ]
    
# datetime(year, month, day, hour, minute, second, microsecond) IN EST!!!
dealer_cutoff_start = datetime(2024, 5, 20, 10, 0, 0, 0)
dealer_cutoff_end = datetime(2024, 5, 21, 23, 0, 0, 0)

# Run fill data for each order in list
all_dealer_fills = multi_order_fills(all_orders)
filtered_df, summary_str, summary_df, exec_price = fills_by_dealer(all_dealer_fills, "ETH", dealer_cutoff_start, dealer_cutoff_end)
# filtered_df, summary_str, summary_df = fills_by_dealer(combined_df)

print(summary_str)
summary_df

In [ ]:
df = pd.read_csv('COINBASE_ETHUSD, 1S (1).csv')
exchange_df = df[['time', 'close']].copy()
exchange_df["time"] = pd.to_datetime(exchange_df["time"])
exchange_df = exchange_df.set_index('time')
exchange_df.index = exchange_df.index.tz_convert("US/Eastern").tz_localize(None)

In [ ]:
order_fills = all_dealer_fills[['Price', 'Quantity']].copy()
merged_df = pd.merge_asof(order_fills, exchange_df, left_index=True, right_index=True, direction='forward')
file_name = '/home/michael_marano_anchorlabs_com/Quant_Research/csv_files/tca_test_eth.csv'
merged_df.to_csv(file_name)

In [ ]:
talos_adv_api = 'ANC8QX7D6WRE'
talos_adv_secret_api = 'nqxfvg2xg9sjpdglo5ul5ydepzpv3h9r'
host_wl = "talostrading.com"
talos_wl = talos_utils.Talos(talos_adv_api, talos_adv_secret_api, host_wl)

symbol = "MKR-USD"
market = "coinbase"

startDate = "2024-05-22T01:01:00.000000Z"
endDate = "2024-05-25T02:01:00.000000Z"

trades = talos_wl.get_market_trades(symbol, market, startDate, endDate, limit=None, after=None, orderBy="-TransactTime")
df = pd.DataFrame(trades)
# talos_wl.get_market_tradesv2(symbol, market)

In [ ]:
def convert_to_et(datetime_string):
    datetime_obj = datetime.strptime(datetime_string, "%Y-%m-%dT%H:%M:%S.%fZ")
    eastern = pytz.timezone('US/Eastern')
    datetime_et = pytz.utc.localize(datetime_obj).astimezone(eastern)
    return datetime_et.replace(tzinfo=None)

# Apply the function to the DataFrame column
df['TransactTime'] = df['TransactTime'].apply(lambda x: convert_to_et(x))
df = df.set_index('TransactTime')
df = df.drop(columns=['Timestamp'])

In [ ]:
df.head()

In [115]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import importlib
from datetime import datetime, timedelta
import requests
import sys
import os
import logging
import time
import pytz

import os
import json
import pandas as pd
from typing import Union
import numpy as np
import sys
import datetime as dt
import logging
import time

#from coinmetrics.api_client import CoinMetricsClient
from dateutil.relativedelta import relativedelta

sys.path.append(os.path.expanduser("~") + "/anchorage/source/python/quant_lib/anchoragequantlib")
import google_sheet_utility as aql_google_sheet_utility
import utils as aql_utils

os.environ["GOOGLE_SHEET_KEY"] = "projects/375663101687/secrets/trading_gsheet_auth_token/versions/1"
google_sheet_key = aql_utils.read_secret(os.environ.get("GOOGLE_SHEET_KEY"))
gsheet_key = json.loads(google_sheet_key)
worksheet_name = "Dealer Arb Calc"
tab_name = "orders"
gsu = aql_google_sheet_utility.GoogleSheetUtility(gsheet_key, worksheet_name)

df = gsu._get_current_sheet_df(tab_name, 0)
orders = df.Orders.tolist()



#Talos
sys.path.append(os.path.expanduser("~") + "/anchorage/source/python/trading/agency_desk/lib/")
import talos_utils

#Coinmetrics
path = os.path.expanduser("~") + "/jupyter-server/transferapi/"
sys.path.append(path)
#import transferapi_lib
key_folder = path + "./key/"
#key_file = key_folder + "coin_metrics.key"
#coinmetrics_key, null_key = transferapi_lib.load_revenue_key(key_file)
#client = CoinMetricsClient(coinmetrics_key, verbose=False)

## Talos Env ##
env = "wl"
##############
talos_adv_api = 'ANC0ASVTOGM0'
talos_adv_secret_api = '8myg2ro5bf2pzfx4m79u9yh501e09nky'
host_wl = "tal-160.prod.talostrading.com"
talos = talos_utils.Talos(talos_adv_api, talos_adv_secret_api, host_wl)   







/home/eddie_akers_anchorlabs_com/.local/lib/python3.9/site-packages/google/auth/_default.py:76 UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds.

AttributeError: 'DataFrame' object has no attribute 'Orders'

In [ ]:
def multi_order_fills(all_orders):
    combined_df = pd.DataFrame()  # Initialize an empty dataframe to store the combined data
    for identifier in all_orders:
        # Call run_function for each identifier and get the dataframe
        try:
            df = talos.get_trade_fills(order_id=identifier)
            df['Amount'] = df['Amount'].astype(float)
            df['Fee'] = df['Fee'].astype(float)
            df['Quantity'] = df['Quantity'].astype(float)
            df["amount_less_fees"] = df["Amount"] - df["Fee"]
            # Concatenate the dataframe to the combined dataframe
            combined_df = pd.concat([combined_df, df], ignore_index=True)
            time.sleep(0.100)
        except Exception as e:
            err_str = f"ERROR during get_trade_fills for orderID {identifier}: {str(e)}\n\n"
            print(err_str)
    if "Timestamp" in combined_df.columns:
        combined_df = combined_df.set_index('Timestamp')
        combined_df = combined_df.sort_index()
    return combined_df

def fills_by_dealer(df, asset, dealer_cutoff_start=None, dealer_cutoff_end=None):
    if dealer_cutoff_start and dealer_cutoff_end is not None:
        filtered_df = df[(df.index >= dealer_cutoff_start) & (df.index < dealer_cutoff_end)]
    else:
        filtered_df = df
    # Group by dealer
    df_by_dealer = filtered_df.groupby("Market")[["amount_less_fees", "Quantity"]].sum().reset_index()
    df_count = filtered_df.groupby("Market")["OrderID"].count().reset_index()
    df_count = df_count.rename(columns={"OrderID": "fill_count"})
    
    pd.options.display.float_format = '{:,.8f}'.format
    amount = filtered_df['amount_less_fees'].sum()
    qty = filtered_df['Quantity'].sum()
    exec_price = amount / qty

    summary_str = f"USD Amount: ${amount:,.2f}\nQuantity: {qty:,.4f} {asset}\nExecution Price: ${exec_price:,.2f}\n"
    summary_df = df_by_dealer.merge(df_count)
    return filtered_df, summary_str, summary_df, exec_price


def vwap(x):
    total_volume = x["amount"].sum()
    return (x['price'] * x["amount"]).sum() / total_volume

def vwap_talos(x):
    total_volume = x["Size"].sum()
    return (x['Price'] * x["Size"]).sum() / total_volume

def add_calculations(df):
    df = df.copy()
    df["vwas"] = (df["Quantity"] * df["bps_spread_to_ref"]).cumsum() / df["Quantity"].cumsum()
    df["cum_qty"] = df["Quantity"].cumsum()
    
    df['rolling_avg_spread'] = df['bps_spread_to_ref'].rolling(len(df), min_periods=1).mean()
    return df

def resample_exchange_trades(exchange_trades):
    # Convert column name to match Talos API output and set as index
    exchange_trades = exchange_trades.rename(columns={"time": "Timestamp"})
    exchange_trades = exchange_trades.set_index('Timestamp').copy()
    
    # Convert timestamp from UTC to EST 
    exchange_trades.index = exchange_trades.index.tz_convert("US/Eastern").tz_localize(None)
    
    # Resample CB trades dataset to VWAP price across each 1S interval. Fill forward where there are no trades for an interval
    exchange_trades_resampled = exchange_trades.resample("500ms").apply(vwap)
    exchange_trades = pd.DataFrame(exchange_trades_resampled, columns=["vwap"])
    exchange_trades = exchange_trades.ffill()
    
    return exchange_trades
    
def calculate_spread_to_cb(all_dealer_fills, exchange_trades):
    # Convert timestamps to int64 (idk if this is needed but I was getting errors)
    all_dealer_fills.index.astype(np.int64) 
    exchange_trades.index.astype(np.int64) 

    # Merge dealer fills df with CB 1s interval VWAP df
    merged_df = pd.merge_asof(all_dealer_fills, exchange_trades, left_index=True, right_index=True, direction='nearest')
    
    # Calculate spread of fills to CB in bps
    cols = ['PriceAllIn', 'vwap', 'Quantity']
    merged_df[cols] = merged_df[cols].apply(pd.to_numeric, errors='coerce', axis=1)
    merged_df["bps_spread_to_ref"] = ((merged_df["PriceAllIn"] - merged_df["vwap"]) / merged_df["vwap"]) * 10000
    merged_df["bps_spread_to_ref"] = pd.to_numeric(merged_df["bps_spread_to_ref"], errors='coerce')

    # Calculate Average and Volume weighted Spread for all fills
    avg_spread_to_ref = merged_df['bps_spread_to_ref'].mean()
    total_volume = merged_df['Quantity'].sum()
    vwas = (merged_df['bps_spread_to_ref'] * merged_df['Quantity']).sum() / total_volume
    summary_str = f"Mean Spread to Reference px: {avg_spread_to_ref:,.4f} bps\nVolume-weighted Avg Spread to Reference px: {vwas:,.4f} bps\n"
   
    return merged_df, summary_str

def datetime_to_str(timestamp):
    utc_datetime = pytz.timezone('US/Eastern').localize(timestamp).astimezone(pytz.utc)
    utc_string = utc_datetime.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
    
    return utc_string

def convert_to_et(datetime_string):
    datetime_obj = datetime.strptime(datetime_string, "%Y-%m-%dT%H:%M:%S.%fZ")
    eastern = pytz.timezone('US/Eastern')
    datetime_et = pytz.utc.localize(datetime_obj).astimezone(eastern)
    
    return datetime_et.replace(tzinfo=None)

def get_exchange_trades_df(symbol, market, startDate, endDate):
    # get talos market data, convert to df
    trades = talos.get_market_trades(symbol, market, startDate, endDate)
    df = pd.DataFrame(trades)
    # set timestamp as index, convert to EST
    df['TransactTime'] = df['TransactTime'].apply(lambda x: convert_to_et(x))
    df = df.drop(columns=['Timestamp'])
    
    return df

def resample_exchange_trades_talos(exchange_trades):
    # Convert column name to match Talos API output and set as index
    exchange_trades = exchange_trades.rename(columns={"TransactTime": "Timestamp"})
    exchange_trades = exchange_trades.set_index('Timestamp').copy()
    
    # Resample CB trades dataset to VWAP price across each 1S interval. Fill forward where there are no trades for an interval
    exchange_trades["Price"] = exchange_trades["Price"].astype(float)
    exchange_trades["Size"] = exchange_trades["Size"].astype(float) 
    exchange_trades_resampled = exchange_trades.resample("500ms").apply(vwap_talos)
    exchange_trades = pd.DataFrame(exchange_trades_resampled, columns=["vwap_talos"])
    exchange_trades = exchange_trades.ffill()
    
    return exchange_trades

def calculate_spread_to_ref(all_dealer_fills, exchange_trades):
    # Merge dealer fills df with CB 1s interval VWAP df
    merged_df = pd.merge_asof(all_dealer_fills, exchange_trades, left_index=True, right_index=True, direction='nearest')
    
    # Calculate spread of fills to CB in bps
    cols = ['PriceAllIn', 'vwap_talos', 'Quantity']
    merged_df[cols] = merged_df[cols].apply(pd.to_numeric, errors='coerce', axis=1)
    merged_df["bps_spread_to_ref"] = ((merged_df["PriceAllIn"] - merged_df["vwap_talos"]) / merged_df["vwap_talos"]) * 10000
    merged_df["bps_spread_to_ref"] = pd.to_numeric(merged_df["bps_spread_to_ref"], errors='coerce')

    # Calculate Average and Volume weighted Spread for all fills
    avg_spread_to_ref = merged_df['bps_spread_to_ref'].mean()
    total_volume = merged_df['Quantity'].sum()
    vwas = (merged_df['bps_spread_to_ref'] * merged_df['Quantity']).sum() / total_volume
    summary_str = f"Mean Spread to Reference px: {avg_spread_to_ref:,.4f} bps\nVolume-weighted Avg Spread to Reference px: {vwas:,.4f} bps\n"
    
    return merged_df, summary_str

['64bf5a2d-6bec-4942-adb4-dab7e7258cca',
 'f3c7aedc-f72c-4192-a64f-d67520c1ff09',
 '03369454-1fbf-4fbf-ac0a-6299af6ccf95',
 'b1fa01b1-7846-4adf-8b78-c1b7f2de4c26',
 '831dcad3-ea98-4d34-bbdd-ae82d748730a',
 'f726aafd-b442-40bf-9a8c-77918fcadb68',
 '16c57372-e1b7-45fe-ac0e-165c15a3f502',
 '8cfed93d-d083-4448-a834-770e58928591',
 '632816e5-13d4-41d1-a1fa-8302cf4a1164',
 'ce1e21ad-693f-4445-9eed-1e9c6f04f2c2',
 'd40ad82c-f8c1-490a-8237-d335765e1380',
 '17105e19-1ad8-4c3b-8b95-a5417674b85b',
 '0a0a3d6e-3ef7-4bb0-8878-df18ebe2c266',
 'ecae2009-01b4-406b-8aec-ddaa00e92faf',
 'be67f70d-ba2a-4dca-a8db-aa13390bc8d4',
 '86453a1d-c71d-4ee6-9bca-5cc72222366a',
 '4e6f4ff3-869a-4a36-b049-3c30ef4809f1',
 '58862d86-5c1e-4e60-9980-4554706c9767',
 'dd65cde6-c89d-43c2-a2e8-ed5e30e344ba',
 'cc6e9b78-df5d-46f6-8924-61d95599818b',
 'b0e349f6-d48f-4d5c-a3cc-cdd1a4093500',
 '2ccb48d4-a243-43e5-a7f1-a3448072286e',
 '541bb447-e698-463b-b1b5-e1682b954a56',
 '0244a577-769b-40a5-bacb-8b1b73be9c1f',
 '2cd77f64-a20a-

USD Amount: $354,223.21
Quantity: 118,103.4383 BTC
Execution Price: $3.00



,Market,amount_less_fees,Quantity,fill_count
0,galaxy,"117,060.56707044",2.00987563,42
1,multileg,"118,052.75693474","118,099.38465862",43
2,wintermute,"119,109.88700000",2.04372418,44


In [ ]:
import pandas as pd

# Initialize an empty list to store DataFrames
all_suborders = []

for order in orders:
    # Fetch the order data
    test_order = talos.get_orders(order)
    
    # Convert the order data to a DataFrame
    test_synthetic = pd.DataFrame(test_order)
    
    # Extract and clean the 'Markets' data for this order
    test_suborders = pd.DataFrame(test_synthetic["Markets"][0]).dropna()
    
    # Add 'OrderID' and 'Timestamp' to each suborder
    test_suborders['OrderID'] = order  # Assuming 'order' contains the OrderID
    test_suborders['Timestamp'] = test_synthetic['Timestamp'].iloc[0]  # Assuming the first timestamp is relevant
    
    # Append the cleaned DataFrame to the list
    all_suborders.append(test_suborders)

# Concatenate all the suborders into a single DataFrame
aggregate_df = pd.concat(all_suborders, ignore_index=True)

# Reorder columns to ensure 'OrderID' and 'Timestamp' are first
columns_order = ['OrderID', 'Timestamp'] + [col for col in aggregate_df.columns if col not in ['OrderID', 'Timestamp']]
aggregate_df = aggregate_df[columns_order]

# Display the final aggregated DataFrame
worksheet_name = "Dealer Arb Calc"
tab_name = "talos_dump"

gsu = aql_google_sheet_utility.GoogleSheetUtility(gsheet_key, worksheet_name)
gsu.dump_current_sheet(tab_name, aggregate_df)

In [ ]:
import pandas as pd

# Assuming df is your DataFrame
df = multi_order_fills(orders)
# Step 1: Filter only relevant columns
relevant_columns = ['Market', 'Side', 'Currency', 'Quantity', 'Price']
df_filtered = df[relevant_columns].copy()

# Step 2: Convert 'Quantity' and 'Price' to numeric values
df_filtered['Quantity'] = pd.to_numeric(df_filtered['Quantity'], errors='coerce')
df_filtered['Price'] = pd.to_numeric(df_filtered['Price'], errors='coerce')

# Step 3: Define the dealers of interest
dealers = ['wintermute', 'galaxy']

# Step 4: Initialize an empty list to store results
results = []

# Step 5: Loop through each dealer
for dealer in dealers:
    # Filter data for the current dealer
    dealer_data = df_filtered[df_filtered['Market'].str.lower() == dealer]

    # Group by Currency and Side to net the positions
    net_positions = dealer_data.groupby(['Currency', 'Side']).agg({
        'Quantity': 'sum',
        'Price': 'mean'
    }).reset_index()

    # Loop through 'BUY' and 'SELL' sides to calculate details
    for side in ['BUY', 'SELL']:
        side_data = net_positions[(net_positions['Currency'] == 'BTC') & 
                                  (net_positions['Side'].str.upper() == side)]

        if not side_data.empty:
            total_qty = side_data['Quantity'].sum()
            weighted_avg_price = dealer_data[dealer_data['Side'].str.upper() == side].apply(
                lambda x: x['Quantity'] * x['Price'], axis=1).sum() / total_qty

            if side == 'BUY':
                buy_qty = total_qty
                sell_qty = buy_qty * weighted_avg_price  # Convert to USD
                buy_asset = 'BTC'
                sell_asset = 'USD'
            else:
                sell_qty = total_qty
                buy_qty = sell_qty * weighted_avg_price  # Convert to USD
                buy_asset = 'USD'
                sell_asset = 'BTC'

            # Append the result for the current dealer and side to the list
            results.append({
                'Dealer': dealer.capitalize(),
                'Side': side,
                'Symbol': 'BTC/USD',
                'Buy QTY': buy_qty,
                'Buy Asset': buy_asset,
                'Sell QTY': sell_qty,
                'Sell Asset': sell_asset,
                'Price': weighted_avg_price
            })

# Step 6: Convert the results list to a DataFrame
result_df = pd.DataFrame(results)

# Display the final DataFrame
display(result_df)

worksheet_name = "Dealer Arb Calc"
tab_name = "talos_dump_sum"

gsu = aql_google_sheet_utility.GoogleSheetUtility(gsheet_key, worksheet_name)
gsu.dump_current_sheet(tab_name, result_df)


,Dealer,Side,Symbol,Buy QTY,Buy Asset,Sell QTY,Sell Asset,Price
0,Wintermute,BUY,BTC/USD,0.62017770,BTC,"36,001.56666960",USD,"58,050.40502037"
1,Wintermute,SELL,BTC/USD,"83,108.37373155",USD,1.42354648,BTC,"58,381.21543565"
2,Galaxy,BUY,BTC/USD,1.40660293,BTC,"82,028.86963949",USD,"58,317.00467131"
3,Galaxy,SELL,BTC/USD,"35,031.69743097",USD,0.60327270,BTC,"58,069.42271874"
